Notes from Stat 849 at UW-Madison: http://www.stat.wisc.edu/courses/st849-bates/lectures/

## Chapter 1: The Gaussian Linear Model
Reponses: Vector-valued random variable, $\mathcal{Y}$. Observed values of the responses, represented by the vector $y$.

Predictors: $X\beta$, where the $n \times p$ matrix $X$ is the *model matrix*, $n$ is the number of obsrvations, and $p$ is the dimension of the *coefficient vector*, $\beta$. The coefficients are *parameters* in the model. We form *estimates* $\hat{\beta}$ of these paramters from the observed data. We assume that $n \geq q$.

$$\mathcal{Y} \sim \text{Multivariate Gaussian}(X\beta_T, \sigma^2I_n)$$
where $\beta_T$ is the "true", but unknown, value of the coefficient vector.